## Evaluation dataset creation from deepeval.synthesizer

documents that use for generate the eval data set

/data

    - "About us.pdf"
    - "contact us.pdf"
    - "delivery.pdf"
    - "payment options.pdf"
    - "returns.pdf"
    - "services.pdf"
    - "testings.pdf"
    - "tracking.pdf"
    - "warranty.pdf"
    - "whatsapp order.pdf"

### Set local evaluation using ollama 
- deepeval set-ollama deepseek-r1:1.5b

In [11]:
from typing import Optional, Tuple, List, Union, Any
from langchain_openai import ChatOpenAI
from langchain_core.messages import AIMessage, BaseMessage
from langchain_core.outputs import ChatResult
from deepeval.models import DeepEvalBaseLLM
from deepeval.metrics.utils import trimAndLoadJson
from langchain_community.callbacks import get_openai_callback

class CustomChatOpenAI(ChatOpenAI):
    format: str = None

    def __init__(self, format: str = None, **kwargs):
        super().__init__(**kwargs)
        self.format = format

    async def _acreate(
        self, messages: List[BaseMessage], **kwargs
    ) -> ChatResult:
        if self.format:
            kwargs["format"] = self.format
        return await super()._acreate(messages, **kwargs)

class OllamaLLM(DeepEvalBaseLLM):
    
    def __init__(
        self,
        model_name: str,
        base_url: str = "http://localhost:11434/v1/",
        json_mode: bool = True,
        temperature: float = 0,
        *args,
        **kwargs
    ):

        self.model_name = model_name
        self.base_url = base_url
        self.json_mode = json_mode
        self.temperature = temperature
        self.args = args
        self.kwargs = kwargs
        super().__init__(model_name)

    def load_model(self) -> CustomChatOpenAI:
        """Load and configure the Ollama model."""
        return CustomChatOpenAI(
            model_name=self.model_name,
            openai_api_key="ollama",  
            base_url=self.base_url,
            format="json" if self.json_mode else None,
            temperature=self.temperature,
            *self.args,
            **self.kwargs,
        )

    def generate(self, prompt: str, schema: Any = None) -> Union[Any, Tuple[str, float]]:

        chat_model = self.load_model()
        with get_openai_callback() as cb:
            res = chat_model.invoke(prompt)
            if schema is not None:
                try:
                    # Try to parse the response using the schema
                    data = trimAndLoadJson(res.content, None)
                    return schema(**data)
                except Exception:
                    # If schema parsing fails, return parsed JSON
                    return trimAndLoadJson(res.content, None)
            return res.content, 0.0

    async def a_generate(self, prompt: str, schema: Any = None) -> Union[Any, Tuple[str, float]]:

        chat_model = self.load_model()
        with get_openai_callback() as cb:
            res = await chat_model.ainvoke(prompt)
            if schema is not None:
                try:
                    # Try to parse the response using the schema
                    data = trimAndLoadJson(res.content, None)
                    return schema(**data)
                except Exception:
                    # If schema parsing fails, return parsed JSON
                    return trimAndLoadJson(res.content, None)
            return res.content, 0.0

    def get_model_name(self) -> str:
        """Get the name of the current model."""
        return self.model_name

    @property
    def __class__(self):
        from deepeval.models import GPTModel  
        return GPTModel

### we can generate evaluation questions from deepeval.synthesizer
I used chatgpt for generating evaluation questions 

In [17]:
questions_answers = [
  {
    "question": "What is TRONIC.LK?",
    "expected answer": "TRONIC.LK is an electronic component and module sourcing company under Sigma Electronics (Pvt) Ltd."
  },
  {
    "question": "What types of products does TRONIC.LK supply?",
    "expected answer": "They supply high quality electronic components, modules, and tools manufactured in China, Taiwan, Hong Kong, Japan, USA, Italy, England, Germany, and Australia."
  },
  {
    "question": "When did TRONIC.LK first introduce Arduino development boards and modules to Sri Lanka?",
    "expected answer": "They introduced them back in 2011 under the company named Lankatronics (Pvt) Ltd."
  },
  {
    "question": "What is the mission of TRONIC.LK?",
    "expected answer": "To introduce high quality components to the local market."
  },
  {
    "question": "What is the vision of TRONIC.LK?",
    "expected answer": "To become the market leader in high quality component sourcing."
  },
  {
    "question": "Where is the TRONIC.LK Electronic Shop located?",
    "expected answer": "At 8, 1/1, Sunethradevi Road, Kohuwala, Nugegoda."
  },
  {
    "question": "What are the operating hours of the TRONIC.LK Electronic Shop?",
    "expected answer": "Open all 7 days from 9:00 AM to 7:00 PM, except on Poya Days and Mercantile Holidays."
  },
  {
    "question": "How can customers check product availability at TRONIC.LK?",
    "expected answer": "They should send an email to check@tronic.lk."
  },
  {
    "question": "What telephone numbers are provided for general inquiries at the Electronic Shop?",
    "expected answer": "0777 854444 and 0114 544566 (Tel/WhatsApp)."
  },
  {
    "question": "What are the operating hours for the TRONIC.LK Online Store?",
    "expected answer": "It is open from Monday to Saturday, 9:00 AM to 6:00 PM, except on Poya Days and Mercantile Holidays."
  },
  {
    "question": "Which email address should be used for Sales Inquiries?",
    "expected answer": "sales@tronic.lk"
  },
  {
    "question": "What is the procedure for special orders?",
    "expected answer": "Customers need to send an email to orders@tronic.lk with their contact details and details of the required items including website links."
  },
  {
    "question": "How does TRONIC.LK provide technical support?",
    "expected answer": "Technical support is provided via their Support Ticket System; inquiries should be sent to support@tronic.lk."
  },
  {
    "question": "Which email address is designated for Corporate Division inquiries?",
    "expected answer": "corp@lankatronics.lk"
  },
  {
    "question": "What is the Corporate Division hotline number?",
    "expected answer": "077 5678 000 (Tel & WhatsApp)."
  },
  {
    "question": "Which division handles R&D and what is their contact email?",
    "expected answer": "Tesla Robotics (Pvt) Ltd. handles R&D; contact them at projects@tesla.lk."
  },
  {
    "question": "How does TRONIC.LK deliver orders across the island?",
    "expected answer": "They deliver items via a courier service with charges based on the weight of the parcel."
  },
  {
    "question": "What is the courier charge for the first kilogram?",
    "expected answer": "Rs. 480 for the first kilogram."
  },
  {
    "question": "How much is charged for each additional kilogram in courier delivery?",
    "expected answer": "Rs. 150 for each additional kilogram."
  },
  {
    "question": "What is the delivery guarantee period for courier services?",
    "expected answer": "Delivery is guaranteed within 1 to 4 working days."
  },
  {
    "question": "How do customers track their parcel?",
    "expected answer": "A tracking number is provided via email, and customers can track their parcel through the courier service's tracking system."
  },
  {
    "question": "What quick delivery option is available for customers in Colombo?",
    "expected answer": "Quick Delivery via Uber or Pickme Flash motorbike/three-wheeler delivery service."
  },
  {
    "question": "How should a customer initiate a fast delivery request using Uber/Pickme?",
    "expected answer": "Place an online order and enter 'Uber / Pickme collection' in the comments box; then follow the provided instructions."
  },
  {
    "question": "What does a customer need to do for a store pickup order?",
    "expected answer": "They should mention 'Store Pickup' in the comment section when placing the order."
  },
  {
    "question": "What is the preferred payment method at TRONIC.LK?",
    "expected answer": "Bank transfers are preferred due to the absence of unnecessary third-party charges."
  },
  {
    "question": "What mobile app can be used for payments that link to multiple bank accounts?",
    "expected answer": "The Frimi app by Nations Trust Bank (NTB)."
  },
  {
    "question": "What processing fee is applied when using a QR payment app like Frimi QR?",
    "expected answer": "A processing fee of 0.5% is applied."
  },
  {
    "question": "What processing fee is charged when using PayHere for card payments?",
    "expected answer": "A processing fee of 2.99% is charged."
  },
  {
    "question": "How can customers opt for easy installment payments?",
    "expected answer": "By requesting an online payment request via PayHere which offers several easy payment options."
  },
  {
    "question": "What is TRONIC.LK’s policy regarding returns and exchanges?",
    "expected answer": "Return/exchange requests (other than warranty claims) are accepted only if items are unopened, sealed, and unused, usually within 7 days of purchase."
  },
  {
    "question": "What is the return procedure for items purchased from the physical store?",
    "expected answer": "Customers must visit the outlet with their bill and the item(s) to be exchanged; the store will take their details and inform them within 7 days."
  },
  {
    "question": "How should online purchase returns be initiated?",
    "expected answer": "By sending an email to info@tronic.lk with contact details, bill date/number, and a list of items to be returned. Additional instructions may follow."
  },
  {
    "question": "Apart from selling products, what additional services does TRONIC.LK offer?",
    "expected answer": "They offer services such as 3D printing, CNC PCB engraving, 3D designing, PCB designing, PCB engraving/etching, PCB production, firmware development, and testing services."
  },
  {
    "question": "How does TRONIC.LK support local technology and innovation?",
    "expected answer": "They share resources like 3D printers and CNC PCB engravers for prototyping purposes at reasonable costs and facilitate limited mass-production orders."
  },
  {
    "question": "What service is available for customers needing 3D design?",
    "expected answer": "In-house 3D modeling and designing for creating enclosures or prototypes, available by contacting projects@lankatronics.lk."
  },
  {
    "question": "How do customers inquire about 3D printing services?",
    "expected answer": "They should send their STL file to projects@lankatronics.lk for a quotation."
  },
  {
    "question": "What is the minimum order quantity for plastic injection moulding?",
    "expected answer": "The minimum order quantity is 1000 pieces."
  },
  {
    "question": "How does TRONIC.LK assist with PCB designing?",
    "expected answer": "They provide in-house PCB designing services; customers can send their project details to projects@lankatronics.lk."
  },
  {
    "question": "What should a customer do to request firmware development services?",
    "expected answer": "They need to send a detailed inquiry with their requirements to projects@lankatronics.lk."
  },
  {
    "question": "What testing services does TRONIC.LK offer for modules and sensors?",
    "expected answer": "They perform tests on modules and sensors, providing reports with photos, videos, diagrams, and source codes, with the cost paid in advance."
  },
  {
    "question": "What happens if a module sold by TRONIC.LK is found to be malfunctioning due to a manufacturing defect?",
    "expected answer": "TRONIC.LK will either offer a refund or a replacement free of charge."
  },
  {
    "question": "What are some conditions that may void the warranty provided by TRONIC.LK?",
    "expected answer": "Warranty may be voided by factors such as burn marks, mechanical damage during transportation, unauthorized modifications, inappropriate installation, misuse, external events, and non-observance of documentation."
  },
  {
    "question": "What special warranty terms apply to inverters and battery chargers?",
    "expected answer": "For inverters, grid power should not be connected to the output and usage should not exceed half the marked output peak wattage. For battery chargers, batteries must be disconnected during charging and continuous charging of weak batteries is not advised."
  }
]